<a href="https://colab.research.google.com/github/vivaria/monte-giga/blob/main/Gigavise_Simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook simulates opening hands to estimate the probability of drawing a winning combination of cards. It is designed for the Gigavise archetype in Edison.



### Monte Carlo Simulations

**Disclaimer**: Monte Carlo simulations can only provide _estimates_. But, this is okay, because if we generate enough simulated hands, the estimates will closely match the exact values. Plus, simulations allow us to write logic that represents complicated, branching situations.

To learn more about Monte Carlo simulations, I highly recommend this [MIT Lecture](https://ocw.mit.edu/courses/6-0002-introduction-to-computational-thinking-and-data-science-fall-2016/5af20311b02eaab959fcdb7ffb5694d3_MIT6_0002F16_lec6.pdf).

### Estimates vs. Exact Probabilities

However, if you're looking for _exact_ probabilities, you've got two options:

1. Test all 2+ million hands, rather than sampling random hands. This takes about 5 minutes, and I've included it as part of this notebook.
2. Use a "multivariate hypergeometric calculator".
    - My favorite calculator used to be [Deck-u-lator](https://deckulator.appspot.com/), as it could support multiple simultaneous combos (just like this simulator can). However, I've been getting weird outputs.
    - If you want to mess around with it, I've set up a quick example of how to use Deck-u-lator for Gigavise. To try it, open Deck-u-lator, then open settings in the top-right corner of the page, then use the "Import" functionality to copy and paste [this text](https://pastebin.com/DeBWkdr6) (hit `raw` then `Ctrl+A`).
    - The test should give you a total probability of 35.6% with 6 cards drawn. However, with my simulator, I'm getting 32.98% instead when testing all combinations. Weird!


### Python, Jupyter, Google Colab, and Git

This page contains **Python code**, stored in a **Jupyter notebook**, run using **Google Colab**, managed using **Git**.

However, you don't need to know all of those things to be able to use this simulator. All you need to know is the basics of Google Colab. You can find out more from [this useful guide](https://mcgrawect.princeton.edu/guides/Google-Colab-Introduction.pdf) from Princeton University.

Or, more simply:

1. Hit "Connect" in the top-right corner.
2. In the top menu, go to "Runtime"
3. Hit "Run All"

For those who are more technically inclined, you can view the source code and revision history on my GitHub account ([vivaria](https://github.com/vivaria) -> [`monte-giga`](https://github.com/vivaria/monte-giga)).

## Simulation Code

The code block below defines the card variables that will be used to build the combos.

In [1]:
# We use variables (`Copy_Plant = "Copy Plant"`) instead of just strings
# ("Copy Plant") because it provides several benefits:
#    - Auto-completion in text editors
#    - Catching typos (thanks to warnings/errors in editors)
#    - Avoiding having to type out the full D.D.R. name every time!
Copy_Plant = "Copy Plant"
Cyber_Valley = "Cyber Valley"
Dandylion = "Dandylion"
Gigaplant = "Gigaplant"
Lonefire_Blossom = "Lonefire Blossom"
Nettles = "Nettles"
Tytannial = "Tytannial, Princess of Camellias"
D_D_R = "D.D.R. - Different Dimension Reincarnation"
Foolish_Burial = "Foolish Burial"
Giant_Trunade = "Giant Trunade"
Gold_Sarcophagus = "Gold Sarcophagus"
Heavy_Storm = "Heavy Storm"
Hidden_Armory = "Hidden Armory"
Miracle_Fertilizer = "Miracle Fertilizer"
One_for_One = "One for One"
Super_Solar_Nutrient = "Super Solar Nutrient"
Supervise = "Supervise"
Raigeki_Break = "Raigeki Break"

The code cell below defines the ratios for each of the above cards.

> - **Note 1**: Anything not defined here will be counted as "Other" for the sake of building hands.
> - **Note 2**: We don't define Upstart Goblin here. Instead, we simplify the calculation by using a 37 card deck instead.

You can edit the values in the cell, then re-run it (and the cells below it) in order to re-run the simulation.

In [2]:
key_card_ratios = {
    Copy_Plant: 1, Nettles: 1,
    Lonefire_Blossom: 2, Gigaplant: 2, Tytannial: 1,
    Foolish_Burial: 1, Dandylion: 2,
    Gold_Sarcophagus: 2,
    Hidden_Armory: 3, Supervise: 3, D_D_R: 2,
    Heavy_Storm: 1, Giant_Trunade: 1,
    Super_Solar_Nutrient: 3,
    Miracle_Fertilizer: 2,
    Raigeki_Break: 3,
    # One_for_One: 1  # No longer played in ps/marcus build
}

The code cell below defines the various combos we consider as a good T1 (or T2 with protection).

In [3]:
# NB: Redundant combos are OK here since we only check for 1 or more combos
combos = [
    (Lonefire_Blossom, Supervise),
    (Lonefire_Blossom, Hidden_Armory, Heavy_Storm),
    (Lonefire_Blossom, Hidden_Armory, Giant_Trunade),
    (Foolish_Burial, Super_Solar_Nutrient),
    (Foolish_Burial, Super_Solar_Nutrient, Hidden_Armory),
    (Foolish_Burial, Super_Solar_Nutrient, Supervise),
    (Foolish_Burial, Miracle_Fertilizer, Hidden_Armory),
    (Foolish_Burial, Miracle_Fertilizer, Supervise),
    (Gold_Sarcophagus, D_D_R, Supervise),
    (Gold_Sarcophagus, Hidden_Armory, Supervise),
    (Gold_Sarcophagus, Hidden_Armory, D_D_R),
    (Gold_Sarcophagus, Hidden_Armory, Hidden_Armory),
    (Gold_Sarcophagus, D_D_R, Dandylion, Super_Solar_Nutrient),
    (Super_Solar_Nutrient, Copy_Plant, Supervise),
    (Super_Solar_Nutrient, Nettles, Supervise),
    (Raigeki_Break, Dandylion, Super_Solar_Nutrient),
    # One for One is no longer used, but we can keep the combos without it affecting anything
    (One_for_One, Dandylion, Super_Solar_Nutrient),
    (One_for_One, Dandylion, Lonefire_Blossom),
    (One_for_One, Super_Solar_Nutrient, Supervise, Gigaplant),
    (One_for_One, Super_Solar_Nutrient, Supervise, Tytannial),
    (One_for_One, Super_Solar_Nutrient, Supervise, Cyber_Valley),
    (One_for_One, Super_Solar_Nutrient, Hidden_Armory, Gigaplant),
    (One_for_One, Super_Solar_Nutrient, Hidden_Armory, Tytannial),
    (One_for_One, Super_Solar_Nutrient, Hidden_Armory, Cyber_Valley),
    (One_for_One, Super_Solar_Nutrient, Hidden_Armory, Lonefire_Blossom),
    (One_for_One, Super_Solar_Nutrient, Hidden_Armory, Nettles),
]

The code cell below defines the functions that we'll use to run the simulation.

In [4]:
from collections import Counter
import itertools
import copy
import random
import sys
import tqdm


def generate_deck(n_upstarts):
    # Generate a list of the key cards
    key_cards = sum([[name] * n for name, n in key_card_ratios.items()], [])  # sum -> combine list of lists
    # Fill the rest of the remaining deck space with "Other" cards
    # The remaining deck space dependencs on the number of Upstart Goblins
    # e.g. 3 Upstarts -> 37 card deck
    deck_size = 40 - n_upstarts
    n_others = deck_size - len(key_cards)
    if n_others < 0:
        raise ValueError(f"Invalid number of Upstart Goblin. {n_upstarts} Upstarts would "
                         f"result in a deck size of {deck_size}, but {min_size} cards are "
                         f"needed to have the necessary key cards.")
    deck = key_cards + (["Other"] * n_others)
    print(f"    Deck generated! ({n_upstarts} Upstarts -> {len(deck)} cards with {n_others} 'Other' cards)")
    return deck


def how_many_combos(hand):
    # This is the simplest way to count the number of combos
    n_combos = sum([is_subset(combo, hand) for combo in combos])

    # However, you can also edit this function to include more complex logic.
    # For example, if you want to test Future Fusion + Alias + Dandylion, you
    # would define the combos (e.g. Future Fusion + Super Solar Nutrient), then
    # you could write something like this instead to test for brick hands:

    # n_combos = 0
    # for combo in combos:
    #    if is_subset(combo, hand):
    #        n_combos += 1
    #        if (Future_Fusion in hand) and (is_subset([Alias], hand) or
    #                                        is_subset([Dandylion, Dandylion], hand)):
    #            n_combos -= 1

    return n_combos


def is_subset(combo, hand):
    # syntax source: https://stackoverflow.com/a/16579133
    # the syntax isn't super clear but at least it works :P
    # we can't use `set` here because hands/combos can contain doubles
    return not Counter(combo) - Counter(hand)


def shuffler(deck, hand_size, n_iter):
    # generator that will generate 'n_iter' hands
    for _ in range(n_iter):
        deck_copy = copy.deepcopy(deck)
        random.shuffle(deck)
        hand = deck[:hand_size]
        yield hand


def monte_carlo(deck, hand_size, n_iter):
    print("    Generating hands...")
    totals = {n: 0 for n in range(len(combos))}
    if n_iter == 'all':
        hands = list(itertools.combinations(deck, hand_size))  # all combinations
        print(f"    All hands -> {len(hands)} hands...")
    else:
        hands = shuffler(deck, hand_size, n_iter)

    print("    Counting combos...")
    for hand in tqdm.tqdm(hands, unit="hand"):
        n_combos = how_many_combos(hand)
        totals[n_combos] += 1
    return totals

The code cell below runs the simulation.

I've written the simulation to compare 10,000 hands vs. all possible hands. That way, you can see roughly how close you get without having to spend the time re-computing all 2.3 million hands every time you want to run the simulation.

In [5]:
upstarts = [3]  # Optional: [0, 1, 2, 3]
n_iters = [10_000, 'all']
hand_size = 6

combos_per_hand = {}
for n_iter in n_iters:
    for n_upstarts in upstarts:
        print(f"\nGenerating {n_iter} sample hands for {n_upstarts} Upstart Goblin...")
        deck = generate_deck(n_upstarts)
        combos_per_hand[n_upstarts] = monte_carlo(deck, hand_size, n_iter)
        print(f"    Combos per hand: {combos_per_hand[n_upstarts]}")
        n_hands = sum(combos_per_hand[n_upstarts].values())
        n_hands_with_zero_combos = combos_per_hand[n_upstarts][0]
        n_hands_with_one_or_more = n_hands - n_hands_with_zero_combos
        percent_chance = (n_hands_with_one_or_more / n_hands) * 100
        print(f"    With {n_upstarts} Upstart Goblins, there is a "
              f"({n_hands_with_one_or_more} / {n_hands} == "
              f"{round(percent_chance, 3)}%) chance of opening at least 1 combo.")


Generating 10000 sample hands for 3 Upstart Goblin...
    Deck generated! (3 Upstarts -> 37 cards with 7 'Other' cards)
    Generating hands...
    Counting combos...


10000it [00:03, 3108.23it/s]


    Combos per hand: {0: 6674, 1: 2519, 2: 525, 3: 230, 4: 44, 5: 4, 6: 4, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0}
    With 3 Upstart Goblins, there is a (3326 / 10000 == 33.26%) chance of opening at least 1 combo.
Generating all sample hands for 3 Upstart Goblin...
    Deck generated! (3 Upstarts -> 37 cards with 7 'Other' cards)
    Generating hands...
    All hands -> 2324784 hands...
    Counting combos...


100%|██████████| 2324784/2324784 [05:05<00:00, 7604.03it/s]


    Combos per hand: {0: 1557963, 1: 573153, 2: 129942, 3: 50331, 4: 10839, 5: 2124, 6: 432, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0}
    With 3 Upstart Goblins, there is a (766821 / 2324784 == 32.985%) chance of opening at least 1 combo.
